# AWS Database Services

## Overview

Amazon Web Services (AWS) provides a comprehensive suite of database services designed to handle different workloads, from traditional relational databases to NoSQL, caching, and data warehousing solutions.

### AWS Database Portfolio

| Service | Type | Use Case |
|---------|------|----------|
| **RDS** | Managed Relational | Traditional OLTP applications |
| **Aurora** | Cloud-Native Relational | High-performance MySQL/PostgreSQL |
| **DynamoDB** | Serverless NoSQL | Low-latency key-value workloads |
| **Redshift** | Data Warehouse | Analytics and BI |
| **ElastiCache** | In-Memory Cache | Session management, real-time analytics |

---

## 1. Amazon RDS (Relational Database Service)

### What is RDS?

Amazon RDS is a **managed relational database service** that simplifies the setup, operation, and scaling of relational databases in the cloud. It handles routine database tasks such as provisioning, patching, backup, recovery, and scaling.

### Supported Database Engines

- **Amazon Aurora** (MySQL & PostgreSQL compatible)
- **MySQL**
- **PostgreSQL**
- **MariaDB**
- **Oracle Database**
- **Microsoft SQL Server**

### Key Features

```
┌─────────────────────────────────────────────────────────────────┐
│                    Amazon RDS Architecture                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌─────────────┐        ┌─────────────┐                        │
│   │   Primary   │───────▶│   Standby   │   Multi-AZ             │
│   │  Instance   │  Sync  │   Replica   │   (Automatic Failover) │
│   └──────┬──────┘        └─────────────┘                        │
│          │                                                       │
│          │ Async                                                 │
│          ▼                                                       │
│   ┌─────────────┐  ┌─────────────┐  ┌─────────────┐             │
│   │ Read Replica│  │ Read Replica│  │ Read Replica│             │
│   │  (Region A) │  │  (Region B) │  │  (Region C) │             │
│   └─────────────┘  └─────────────┘  └─────────────┘             │
│                                                                  │
│   ┌─────────────────────────────────────────────────┐           │
│   │              Automated Backups                   │           │
│   │  • Daily snapshots (35-day retention)           │           │
│   │  • Transaction logs (5-minute granularity)      │           │
│   │  • Point-in-time recovery                       │           │
│   └─────────────────────────────────────────────────┘           │
└─────────────────────────────────────────────────────────────────┘
```

### RDS Instance Classes

| Class | Description | Use Case |
|-------|-------------|----------|
| **db.t3/t4g** | Burstable | Dev/test, light workloads |
| **db.m5/m6g** | General Purpose | Balanced compute/memory |
| **db.r5/r6g** | Memory Optimized | Memory-intensive workloads |
| **db.x1/x2g** | Memory Optimized | Large in-memory databases |

In [ ]:
# Example: RDS Configuration with boto3
import boto3

def create_rds_instance():
    """
    Create an RDS MySQL instance with Multi-AZ deployment
    """
    rds_client = boto3.client('rds', region_name='us-east-1')
    
    response = rds_client.create_db_instance(
        DBInstanceIdentifier='my-production-db',
        DBInstanceClass='db.r5.large',
        Engine='mysql',
        EngineVersion='8.0.32',
        MasterUsername='admin',
        MasterUserPassword='SecurePassword123!',  # Use Secrets Manager in production
        AllocatedStorage=100,
        StorageType='gp3',
        StorageEncrypted=True,
        MultiAZ=True,  # Enable Multi-AZ for high availability
        AutoMinorVersionUpgrade=True,
        BackupRetentionPeriod=7,
        DeletionProtection=True,
        EnablePerformanceInsights=True,
        PerformanceInsightsRetentionPeriod=7,
        VpcSecurityGroupIds=['sg-xxxxxxxx'],
        DBSubnetGroupName='my-db-subnet-group',
        Tags=[
            {'Key': 'Environment', 'Value': 'Production'},
            {'Key': 'Application', 'Value': 'MyApp'}
        ]
    )
    
    return response

# Create read replica for read scaling
def create_read_replica():
    rds_client = boto3.client('rds', region_name='us-east-1')
    
    response = rds_client.create_db_instance_read_replica(
        DBInstanceIdentifier='my-production-db-replica',
        SourceDBInstanceIdentifier='my-production-db',
        DBInstanceClass='db.r5.large',
        AvailabilityZone='us-east-1b',
        PubliclyAccessible=False,
        EnablePerformanceInsights=True
    )
    
    return response

---

## 2. Amazon Aurora

### What is Aurora?

Amazon Aurora is a **cloud-native relational database** that combines the performance and availability of high-end commercial databases with the simplicity and cost-effectiveness of open-source databases. It's fully compatible with MySQL and PostgreSQL.

### Aurora Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                     Aurora Storage Architecture                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   ┌─────────────┐     ┌─────────────┐     ┌─────────────┐           │
│   │   Writer    │     │   Reader    │     │   Reader    │           │
│   │  Instance   │     │  Instance   │     │  Instance   │           │
│   └──────┬──────┘     └──────┬──────┘     └──────┬──────┘           │
│          │                   │                   │                   │
│          └───────────────────┼───────────────────┘                   │
│                              │                                       │
│                              ▼                                       │
│   ┌─────────────────────────────────────────────────────────────┐   │
│   │              Shared Distributed Storage Layer                │   │
│   │                                                              │   │
│   │   ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐          │   │
│   │   │ AZ1 │ │ AZ1 │ │ AZ2 │ │ AZ2 │ │ AZ3 │ │ AZ3 │          │   │
│   │   │Copy1│ │Copy2│ │Copy1│ │Copy2│ │Copy1│ │Copy2│          │   │
│   │   └─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘          │   │
│   │                                                              │   │
│   │   6 copies across 3 AZs = 99.99% durability                 │   │
│   └─────────────────────────────────────────────────────────────┘   │
│                                                                      │
│   Auto-scaling storage: 10GB → 128TB                                │
└─────────────────────────────────────────────────────────────────────┘
```

### Aurora vs Standard RDS

| Feature | Aurora | Standard RDS |
|---------|--------|-------------|
| **Performance** | 5x MySQL, 3x PostgreSQL | Standard engine performance |
| **Storage** | Auto-scaling to 128TB | Manual provisioning |
| **Replication** | Up to 15 read replicas | Up to 5 read replicas |
| **Failover** | < 30 seconds | 1-2 minutes |
| **Backup** | Continuous to S3 | Daily snapshots |
| **Cost** | ~20% more expensive | Standard pricing |

### Aurora Serverless v2

```
┌────────────────────────────────────────────────────────┐
│              Aurora Serverless v2 Scaling               │
├────────────────────────────────────────────────────────┤
│                                                         │
│   Capacity (ACUs)                                       │
│        │                                                │
│    128 ┤                    ╭──────╮                    │
│        │                   ╱        ╲                   │
│     64 ┤                  ╱          ╲                  │
│        │        ╭───────╯            ╰────╮            │
│     16 ┤       ╱                          ╲            │
│        │  ────╯                            ╰────       │
│    0.5 ┼──────────────────────────────────────────     │
│        └────────────────────────────────────────▶      │
│                        Time                             │
│                                                         │
│   • Scales in increments of 0.5 ACUs                   │
│   • Min: 0.5 ACU, Max: 128 ACUs                        │
│   • Pay per ACU-hour used                              │
└────────────────────────────────────────────────────────┘
```

In [ ]:
# Example: Aurora Serverless v2 Configuration
import boto3

def create_aurora_serverless_v2_cluster():
    """
    Create an Aurora Serverless v2 cluster with auto-scaling
    """
    rds_client = boto3.client('rds', region_name='us-east-1')
    
    # Create Aurora cluster
    cluster_response = rds_client.create_db_cluster(
        DBClusterIdentifier='my-aurora-serverless-cluster',
        Engine='aurora-postgresql',
        EngineVersion='15.4',
        EngineMode='provisioned',  # Serverless v2 uses provisioned mode
        MasterUsername='admin',
        MasterUserPassword='SecurePassword123!',
        DatabaseName='myapp',
        StorageEncrypted=True,
        EnableCloudwatchLogsExports=['postgresql'],
        ServerlessV2ScalingConfiguration={
            'MinCapacity': 0.5,  # Minimum ACUs
            'MaxCapacity': 64    # Maximum ACUs
        },
        VpcSecurityGroupIds=['sg-xxxxxxxx'],
        DBSubnetGroupName='my-db-subnet-group',
        DeletionProtection=True
    )
    
    # Create Serverless v2 writer instance
    writer_response = rds_client.create_db_instance(
        DBInstanceIdentifier='my-aurora-writer',
        DBInstanceClass='db.serverless',  # Serverless v2 instance class
        DBClusterIdentifier='my-aurora-serverless-cluster',
        Engine='aurora-postgresql',
        EnablePerformanceInsights=True
    )
    
    # Create Serverless v2 reader instance
    reader_response = rds_client.create_db_instance(
        DBInstanceIdentifier='my-aurora-reader',
        DBInstanceClass='db.serverless',
        DBClusterIdentifier='my-aurora-serverless-cluster',
        Engine='aurora-postgresql',
        EnablePerformanceInsights=True
    )
    
    return cluster_response, writer_response, reader_response

# Aurora Global Database for cross-region disaster recovery
def create_global_database():
    rds_client = boto3.client('rds', region_name='us-east-1')
    
    response = rds_client.create_global_cluster(
        GlobalClusterIdentifier='my-global-aurora',
        SourceDBClusterIdentifier='my-aurora-serverless-cluster',
        Engine='aurora-postgresql'
    )
    
    return response

---

## 3. Amazon DynamoDB

### What is DynamoDB?

Amazon DynamoDB is a **fully managed, serverless NoSQL database** that delivers single-digit millisecond performance at any scale. It's designed for applications that need consistent, fast performance with automatic scaling.

### DynamoDB Data Model

```
┌─────────────────────────────────────────────────────────────────────┐
│                     DynamoDB Table Structure                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   Table: Orders                                                      │
│   ┌─────────────────────────────────────────────────────────────┐   │
│   │ Partition Key (PK) │ Sort Key (SK)    │ Attributes...       │   │
│   ├────────────────────┼──────────────────┼─────────────────────┤   │
│   │ USER#123           │ ORDER#2024-001   │ total: 150.00       │   │
│   │ USER#123           │ ORDER#2024-002   │ total: 89.99        │   │
│   │ USER#123           │ PROFILE          │ name: "John Doe"    │   │
│   │ USER#456           │ ORDER#2024-001   │ total: 250.00       │   │
│   └─────────────────────────────────────────────────────────────┘   │
│                                                                      │
│   Primary Key = Partition Key + Sort Key (optional)                 │
│                                                                      │
│   ┌─────────────────────────────────────────────────────────────┐   │
│   │              Global Secondary Index (GSI)                    │   │
│   │                                                              │   │
│   │   GSI: OrdersByDate                                         │   │
│   │   PK: order_date    SK: order_id                            │   │
│   │   Enables queries like: "Get all orders from 2024-01-15"    │   │
│   └─────────────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────────────┘
```

### Capacity Modes

| Mode | Description | Best For |
|------|-------------|----------|
| **On-Demand** | Pay-per-request | Unpredictable workloads |
| **Provisioned** | Reserved capacity | Predictable, steady workloads |
| **Reserved Capacity** | 1-3 year commitment | Cost optimization |

### DynamoDB Features

- **DynamoDB Streams**: Capture table changes for event-driven architectures
- **Global Tables**: Multi-region, multi-active replication
- **DAX (DynamoDB Accelerator)**: In-memory cache for microsecond latency
- **TTL (Time to Live)**: Automatic item expiration
- **Point-in-Time Recovery**: 35-day backup window

In [ ]:
# Example: DynamoDB Operations with boto3
import boto3
from decimal import Decimal
from boto3.dynamodb.conditions import Key, Attr

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

def create_dynamodb_table():
    """
    Create a DynamoDB table with GSI for efficient querying
    """
    table = dynamodb.create_table(
        TableName='Orders',
        KeySchema=[
            {'AttributeName': 'pk', 'KeyType': 'HASH'},   # Partition key
            {'AttributeName': 'sk', 'KeyType': 'RANGE'}   # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'pk', 'AttributeType': 'S'},
            {'AttributeName': 'sk', 'AttributeType': 'S'},
            {'AttributeName': 'gsi1pk', 'AttributeType': 'S'},
            {'AttributeName': 'gsi1sk', 'AttributeType': 'S'}
        ],
        GlobalSecondaryIndexes=[
            {
                'IndexName': 'GSI1',
                'KeySchema': [
                    {'AttributeName': 'gsi1pk', 'KeyType': 'HASH'},
                    {'AttributeName': 'gsi1sk', 'KeyType': 'RANGE'}
                ],
                'Projection': {'ProjectionType': 'ALL'}
            }
        ],
        BillingMode='PAY_PER_REQUEST',  # On-demand mode
        StreamSpecification={
            'StreamEnabled': True,
            'StreamViewType': 'NEW_AND_OLD_IMAGES'
        },
        Tags=[
            {'Key': 'Environment', 'Value': 'Production'}
        ]
    )
    
    # Wait for table creation
    table.meta.client.get_waiter('table_exists').wait(TableName='Orders')
    
    return table

def single_table_design_operations():
    """
    Demonstrate single-table design pattern
    """
    table = dynamodb.Table('Orders')
    
    # Put user profile
    table.put_item(Item={
        'pk': 'USER#123',
        'sk': 'PROFILE',
        'name': 'John Doe',
        'email': 'john@example.com',
        'gsi1pk': 'USER',
        'gsi1sk': 'john@example.com'
    })
    
    # Put order
    table.put_item(Item={
        'pk': 'USER#123',
        'sk': 'ORDER#2024-001',
        'total': Decimal('150.00'),
        'status': 'COMPLETED',
        'gsi1pk': 'ORDER#2024-01-15',
        'gsi1sk': 'ORDER#2024-001'
    })
    
    # Query user's orders
    response = table.query(
        KeyConditionExpression=Key('pk').eq('USER#123') & Key('sk').begins_with('ORDER#'),
        ScanIndexForward=False  # Descending order
    )
    
    return response['Items']

# Batch operations for high throughput
def batch_write_items(items):
    table = dynamodb.Table('Orders')
    
    with table.batch_writer() as batch:
        for item in items:
            batch.put_item(Item=item)
    
    return True

---

## 4. Amazon Redshift

### What is Redshift?

Amazon Redshift is a **fully managed, petabyte-scale data warehouse** service designed for online analytical processing (OLAP) workloads. It uses columnar storage and massively parallel processing (MPP) to deliver fast query performance.

### Redshift Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                     Redshift Cluster Architecture                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   ┌─────────────────────────────────────────────────────────────┐   │
│   │                      Leader Node                             │   │
│   │  • Query coordination & optimization                         │   │
│   │  • Query plan generation                                     │   │
│   │  • Result aggregation                                        │   │
│   └──────────────────────────┬──────────────────────────────────┘   │
│                              │                                       │
│          ┌───────────────────┼───────────────────┐                  │
│          ▼                   ▼                   ▼                  │
│   ┌────────────┐     ┌────────────┐     ┌────────────┐             │
│   │ Compute    │     │ Compute    │     │ Compute    │             │
│   │ Node 1     │     │ Node 2     │     │ Node N     │             │
│   │            │     │            │     │            │             │
│   │ ┌────────┐ │     │ ┌────────┐ │     │ ┌────────┐ │             │
│   │ │Slice 1 │ │     │ │Slice 1 │ │     │ │Slice 1 │ │             │
│   │ ├────────┤ │     │ ├────────┤ │     │ ├────────┤ │             │
│   │ │Slice 2 │ │     │ │Slice 2 │ │     │ │Slice 2 │ │             │
│   │ └────────┘ │     │ └────────┘ │     │ └────────┘ │             │
│   └────────────┘     └────────────┘     └────────────┘             │
│                                                                      │
│   Columnar Storage: Only reads columns needed for query             │
│   Compression: Up to 10x compression ratio                          │
└─────────────────────────────────────────────────────────────────────┘
```

### Redshift Serverless vs Provisioned

| Feature | Redshift Serverless | Redshift Provisioned |
|---------|---------------------|----------------------|
| **Scaling** | Automatic | Manual node management |
| **Pricing** | Pay per RPU-hour | Pay per node-hour |
| **Management** | Zero administration | Cluster maintenance |
| **Best For** | Variable workloads | Steady, predictable loads |

### Distribution Styles

| Style | Description | Use Case |
|-------|-------------|----------|
| **KEY** | Distribute by column value | Large fact tables with joins |
| **EVEN** | Round-robin distribution | Tables without joins |
| **ALL** | Copy to all nodes | Small dimension tables |
| **AUTO** | Redshift chooses | Default (recommended) |

In [ ]:
# Example: Redshift Operations
import boto3
import redshift_connector  # pip install redshift-connector

def create_redshift_serverless_workgroup():
    """
    Create a Redshift Serverless workgroup
    """
    client = boto3.client('redshift-serverless', region_name='us-east-1')
    
    # Create namespace (database and users)
    namespace_response = client.create_namespace(
        namespaceName='my-analytics-namespace',
        adminUsername='admin',
        adminUserPassword='SecurePassword123!',
        dbName='analytics',
        iamRoles=['arn:aws:iam::123456789012:role/RedshiftS3Access'],
        logExports=['userlog', 'connectionlog', 'useractivitylog']
    )
    
    # Create workgroup (compute resources)
    workgroup_response = client.create_workgroup(
        workgroupName='my-analytics-workgroup',
        namespaceName='my-analytics-namespace',
        baseCapacity=32,  # RPUs (8-512)
        enhancedVpcRouting=True,
        securityGroupIds=['sg-xxxxxxxx'],
        subnetIds=['subnet-xxxxxxxx', 'subnet-yyyyyyyy']
    )
    
    return namespace_response, workgroup_response

# Redshift query examples
def redshift_analytics_queries():
    """
    Example analytics queries with best practices
    """
    queries = {
        # Create table with optimal distribution and sort keys
        'create_table': '''
            CREATE TABLE sales_fact (
                sale_id BIGINT IDENTITY(1,1),
                customer_id INTEGER NOT NULL,
                product_id INTEGER NOT NULL,
                sale_date DATE NOT NULL,
                quantity INTEGER,
                amount DECIMAL(12,2),
                region VARCHAR(50)
            )
            DISTSTYLE KEY
            DISTKEY (customer_id)
            SORTKEY (sale_date);
        ''',
        
        # Load data from S3 using COPY command
        'load_data': '''
            COPY sales_fact
            FROM 's3://my-bucket/sales/'
            IAM_ROLE 'arn:aws:iam::123456789012:role/RedshiftS3Access'
            FORMAT AS PARQUET;
        ''',
        
        # Analytical query with window functions
        'analytics_query': '''
            WITH monthly_sales AS (
                SELECT 
                    DATE_TRUNC('month', sale_date) as month,
                    region,
                    SUM(amount) as total_sales,
                    COUNT(DISTINCT customer_id) as unique_customers
                FROM sales_fact
                WHERE sale_date >= DATEADD(year, -1, CURRENT_DATE)
                GROUP BY 1, 2
            )
            SELECT 
                month,
                region,
                total_sales,
                unique_customers,
                LAG(total_sales) OVER (PARTITION BY region ORDER BY month) as prev_month_sales,
                (total_sales - LAG(total_sales) OVER (PARTITION BY region ORDER BY month)) 
                    / NULLIF(LAG(total_sales) OVER (PARTITION BY region ORDER BY month), 0) * 100 as growth_pct
            FROM monthly_sales
            ORDER BY month DESC, region;
        '''
    }
    
    return queries

---

## 5. Amazon ElastiCache

### What is ElastiCache?

Amazon ElastiCache is a **fully managed in-memory caching service** that supports Redis and Memcached. It improves application performance by retrieving data from fast, managed, in-memory caches instead of slower disk-based databases.

### ElastiCache Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                  ElastiCache Caching Patterns                        │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   Cache-Aside (Lazy Loading)                                        │
│   ┌─────────────┐                                                   │
│   │ Application │                                                   │
│   └──────┬──────┘                                                   │
│          │                                                          │
│    1. Check Cache                                                   │
│          │                                                          │
│          ▼                                                          │
│   ┌─────────────┐   Cache Miss    ┌─────────────┐                  │
│   │ ElastiCache │───────────────▶ │  Database   │                  │
│   │   (Redis)   │◀─────────────── │   (RDS)     │                  │
│   └─────────────┘   3. Write to   └─────────────┘                  │
│          │             Cache           │                            │
│          │                        2. Read from DB                   │
│          ▼                                                          │
│   Cache Hit: Return data directly                                   │
│                                                                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   Write-Through                                                     │
│   ┌─────────────┐                                                   │
│   │ Application │                                                   │
│   └──────┬──────┘                                                   │
│          │                                                          │
│          │ Write                                                    │
│          ▼                                                          │
│   ┌─────────────┐    Sync Write   ┌─────────────┐                  │
│   │ ElastiCache │───────────────▶ │  Database   │                  │
│   │   (Redis)   │                 │   (RDS)     │                  │
│   └─────────────┘                 └─────────────┘                  │
│                                                                      │
│   Data always written to cache and database simultaneously         │
└─────────────────────────────────────────────────────────────────────┘
```

### Redis vs Memcached

| Feature | Redis | Memcached |
|---------|-------|----------|
| **Data Structures** | Strings, Lists, Sets, Hashes, Sorted Sets | Key-Value only |
| **Persistence** | RDB + AOF | None |
| **Replication** | Multi-AZ with failover | None |
| **Clustering** | Cluster mode | Multi-threaded |
| **Pub/Sub** | Yes | No |
| **Use Case** | Complex caching, sessions, leaderboards | Simple caching |

In [ ]:
# Example: ElastiCache Redis Operations
import boto3
import redis
import json
from functools import wraps
from datetime import timedelta

def create_elasticache_redis_cluster():
    """
    Create an ElastiCache Redis cluster with cluster mode enabled
    """
    client = boto3.client('elasticache', region_name='us-east-1')
    
    response = client.create_replication_group(
        ReplicationGroupId='my-redis-cluster',
        ReplicationGroupDescription='Production Redis Cluster',
        Engine='redis',
        EngineVersion='7.0',
        CacheNodeType='cache.r6g.large',
        NumNodeGroups=3,  # Number of shards
        ReplicasPerNodeGroup=2,  # Read replicas per shard
        AutomaticFailoverEnabled=True,
        MultiAZEnabled=True,
        AtRestEncryptionEnabled=True,
        TransitEncryptionEnabled=True,
        CacheSubnetGroupName='my-cache-subnet-group',
        SecurityGroupIds=['sg-xxxxxxxx'],
        SnapshotRetentionLimit=7,
        SnapshotWindow='05:00-06:00',
        Tags=[
            {'Key': 'Environment', 'Value': 'Production'}
        ]
    )
    
    return response

# Redis caching patterns
class CacheService:
    def __init__(self, redis_host, redis_port=6379):
        self.redis_client = redis.Redis(
            host=redis_host,
            port=redis_port,
            decode_responses=True,
            ssl=True
        )
    
    def cache_aside(self, key, fetch_func, ttl=300):
        """
        Cache-aside pattern: Check cache first, fetch from DB if miss
        """
        # Try to get from cache
        cached_data = self.redis_client.get(key)
        if cached_data:
            return json.loads(cached_data)
        
        # Fetch from database
        data = fetch_func()
        
        # Store in cache
        self.redis_client.setex(key, ttl, json.dumps(data))
        
        return data
    
    def session_management(self, session_id, user_data, ttl=3600):
        """
        Store user session in Redis
        """
        key = f"session:{session_id}"
        self.redis_client.hset(key, mapping=user_data)
        self.redis_client.expire(key, ttl)
    
    def leaderboard(self, leaderboard_name):
        """
        Real-time leaderboard using sorted sets
        """
        # Add score
        self.redis_client.zadd(leaderboard_name, {'player1': 1000, 'player2': 850})
        
        # Get top 10
        return self.redis_client.zrevrange(leaderboard_name, 0, 9, withscores=True)
    
    def rate_limiter(self, user_id, limit=100, window=60):
        """
        Rate limiting using sliding window
        """
        key = f"rate_limit:{user_id}"
        current = self.redis_client.incr(key)
        
        if current == 1:
            self.redis_client.expire(key, window)
        
        return current <= limit

---

## 6. Use Case Comparison

### Choosing the Right AWS Database

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     AWS Database Decision Tree                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   What type of workload?                                                │
│          │                                                               │
│          ├──▶ OLTP (Transactional)                                      │
│          │         │                                                     │
│          │         ├──▶ Need relational/SQL?                            │
│          │         │         │                                           │
│          │         │         ├──▶ YES: Need high performance?           │
│          │         │         │         │                                 │
│          │         │         │         ├──▶ YES ──▶ Aurora              │
│          │         │         │         └──▶ NO ───▶ RDS                 │
│          │         │         │                                           │
│          │         │         └──▶ NO: Key-value access?                 │
│          │         │                   │                                 │
│          │         │                   └──▶ YES ──▶ DynamoDB            │
│          │         │                                                     │
│          │         └──▶ Need sub-millisecond latency?                   │
│          │                   │                                           │
│          │                   └──▶ YES ──▶ ElastiCache + Database        │
│          │                                                               │
│          └──▶ OLAP (Analytical)                                         │
│                    │                                                     │
│                    └──▶ Redshift                                        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Use Case Matrix

| Use Case | Primary Service | Secondary Service |
|----------|-----------------|-------------------|
| E-commerce catalog | DynamoDB | ElastiCache |
| Financial transactions | Aurora | RDS |
| Gaming leaderboards | ElastiCache (Redis) | DynamoDB |
| IoT time-series | DynamoDB / Timestream | - |
| Business intelligence | Redshift | - |
| Session management | ElastiCache | DynamoDB |
| Social media feeds | DynamoDB | ElastiCache |
| ERP/CRM systems | Aurora/RDS | - |
| Real-time analytics | Redshift Streaming | ElastiCache |

---

## 7. Pricing Models

### Cost Comparison Overview

| Service | Pricing Model | Key Cost Factors |
|---------|---------------|------------------|
| **RDS** | Instance hours + Storage + I/O | Instance type, Multi-AZ, storage |
| **Aurora** | Instance hours + Storage + I/O | ACUs (Serverless), storage auto-scaling |
| **DynamoDB** | On-Demand: RRU/WRU or Provisioned: RCU/WCU | Read/write capacity, storage, GSIs |
| **Redshift** | Node hours or RPU-hours (Serverless) | Node type, data scanned |
| **ElastiCache** | Node hours | Node type, number of nodes |

### Cost Optimization Strategies

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Cost Optimization Strategies                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   RDS / Aurora                                                       │
│   ├── Reserved Instances (1-3 year) ─────────▶ Up to 69% savings    │
│   ├── Right-size instances ──────────────────▶ Match workload       │
│   ├── Aurora Serverless v2 ──────────────────▶ Variable workloads   │
│   └── Stop dev/test databases ───────────────▶ Non-prod hours       │
│                                                                      │
│   DynamoDB                                                           │
│   ├── On-Demand for unpredictable ───────────▶ No over-provisioning │
│   ├── Reserved Capacity ─────────────────────▶ Up to 77% savings    │
│   ├── Use TTL for expiring data ─────────────▶ Reduce storage       │
│   └── Optimize GSI projections ──────────────▶ Minimize RCUs        │
│                                                                      │
│   Redshift                                                           │
│   ├── Reserved Nodes ────────────────────────▶ Up to 75% savings    │
│   ├── Pause/Resume clusters ─────────────────▶ Off-hours savings    │
│   ├── Redshift Serverless ───────────────────▶ Intermittent queries │
│   └── Concurrency Scaling ───────────────────▶ Burst capacity       │
│                                                                      │
│   ElastiCache                                                        │
│   ├── Reserved Nodes ────────────────────────▶ Up to 55% savings    │
│   ├── Right-size node types ─────────────────▶ Memory optimization  │
│   └── Data tiering (Redis) ──────────────────▶ SSD for cold data    │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### Estimated Monthly Costs (US-East-1)

| Service | Small Workload | Medium Workload | Large Workload |
|---------|----------------|-----------------|----------------|
| **RDS (MySQL)** | ~$50 (db.t3.micro) | ~$300 (db.m5.large) | ~$1,500 (db.r5.2xlarge) |
| **Aurora Serverless v2** | ~$100 (0.5-4 ACU) | ~$500 (4-32 ACU) | ~$2,000+ (32-128 ACU) |
| **DynamoDB On-Demand** | ~$25 (1M reads/writes) | ~$250 (10M reads/writes) | ~$2,500+ (100M reads/writes) |
| **Redshift Serverless** | ~$90 (8 RPU) | ~$360 (32 RPU) | ~$2,000+ (128+ RPU) |
| **ElastiCache Redis** | ~$25 (cache.t3.micro) | ~$150 (cache.m5.large) | ~$600 (cache.r5.2xlarge) |

In [ ]:
# Cost estimation helper functions
def estimate_dynamodb_cost(read_units_per_month, write_units_per_month, storage_gb, mode='on_demand'):
    """
    Estimate DynamoDB monthly cost (US-East-1 pricing)
    """
    if mode == 'on_demand':
        # On-demand pricing
        read_cost = (read_units_per_month / 1_000_000) * 0.25  # $0.25 per million RRUs
        write_cost = (write_units_per_month / 1_000_000) * 1.25  # $1.25 per million WRUs
    else:
        # Provisioned pricing (assuming 720 hours/month)
        rcu_needed = read_units_per_month / (720 * 3600)  # RCU per second
        wcu_needed = write_units_per_month / (720 * 3600)  # WCU per second
        read_cost = rcu_needed * 0.00013 * 720  # $0.00013 per RCU per hour
        write_cost = wcu_needed * 0.00065 * 720  # $0.00065 per WCU per hour
    
    storage_cost = storage_gb * 0.25  # $0.25 per GB/month
    
    total = read_cost + write_cost + storage_cost
    
    return {
        'read_cost': round(read_cost, 2),
        'write_cost': round(write_cost, 2),
        'storage_cost': round(storage_cost, 2),
        'total_monthly_cost': round(total, 2)
    }

def estimate_rds_cost(instance_class, storage_gb, multi_az=False, reserved=False):
    """
    Estimate RDS monthly cost (US-East-1 pricing, MySQL)
    """
    # Approximate hourly rates
    instance_rates = {
        'db.t3.micro': 0.017,
        'db.t3.small': 0.034,
        'db.t3.medium': 0.068,
        'db.m5.large': 0.171,
        'db.m5.xlarge': 0.342,
        'db.r5.large': 0.24,
        'db.r5.xlarge': 0.48,
        'db.r5.2xlarge': 0.96
    }
    
    hourly_rate = instance_rates.get(instance_class, 0.1)
    
    if multi_az:
        hourly_rate *= 2
    
    if reserved:
        hourly_rate *= 0.35  # ~65% discount for 3-year reserved
    
    instance_cost = hourly_rate * 720  # 720 hours/month
    storage_cost = storage_gb * 0.115  # gp2 storage
    
    return {
        'instance_cost': round(instance_cost, 2),
        'storage_cost': round(storage_cost, 2),
        'total_monthly_cost': round(instance_cost + storage_cost, 2)
    }

# Example usage
print("DynamoDB Cost Estimate:")
print(estimate_dynamodb_cost(
    read_units_per_month=10_000_000,
    write_units_per_month=5_000_000,
    storage_gb=50,
    mode='on_demand'
))

print("\nRDS Cost Estimate:")
print(estimate_rds_cost(
    instance_class='db.m5.large',
    storage_gb=100,
    multi_az=True,
    reserved=False
))

---

## 8. Key Takeaways

### Summary

| Service | Best For | Key Advantage |
|---------|----------|---------------|
| **RDS** | Traditional OLTP apps | Familiar SQL, managed operations |
| **Aurora** | High-performance relational | 5x MySQL performance, auto-scaling storage |
| **DynamoDB** | Serverless, low-latency NoSQL | Infinite scale, single-digit ms latency |
| **Redshift** | Data warehousing & analytics | Petabyte-scale, columnar storage |
| **ElastiCache** | Caching & real-time data | Sub-millisecond latency, Redis features |

### Best Practices Checklist

```
✅ Security
   □ Enable encryption at rest and in transit
   □ Use VPC and security groups for network isolation
   □ Store credentials in AWS Secrets Manager
   □ Enable IAM database authentication where supported

✅ High Availability
   □ Enable Multi-AZ for RDS/Aurora
   □ Use DynamoDB Global Tables for multi-region
   □ Configure ElastiCache with replicas
   □ Implement proper backup and recovery procedures

✅ Performance
   □ Right-size instances based on workload metrics
   □ Use read replicas for read-heavy workloads
   □ Implement caching layer with ElastiCache
   □ Monitor with Performance Insights and CloudWatch

✅ Cost Optimization
   □ Use Reserved Instances/Capacity for steady workloads
   □ Consider Serverless options for variable workloads
   □ Implement data lifecycle policies (TTL, tiering)
   □ Stop non-production resources during off-hours
```

### Architecture Pattern: Multi-Database Stack

```
┌─────────────────────────────────────────────────────────────────────┐
│                  Modern Application Database Stack                   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   ┌─────────────────────────────────────────────────────────────┐   │
│   │                      Application Layer                       │   │
│   └──────────────────────────┬──────────────────────────────────┘   │
│                              │                                       │
│   ┌──────────────────────────┼──────────────────────────┐           │
│   │                          ▼                          │           │
│   │  ┌─────────────┐  ┌─────────────┐  ┌────────────┐  │           │
│   │  │ ElastiCache │  │   Aurora    │  │  DynamoDB  │  │ Hot Data  │
│   │  │   (Redis)   │  │   (OLTP)    │  │  (NoSQL)   │  │           │
│   │  │             │  │             │  │            │  │           │
│   │  │ Sessions    │  │ Transactions│  │ User Data  │  │           │
│   │  │ Cache       │  │ Core Data   │  │ Events     │  │           │
│   │  └─────────────┘  └──────┬──────┘  └─────┬──────┘  │           │
│   └──────────────────────────┼───────────────┼──────────┘           │
│                              │               │                       │
│                              ▼               ▼                       │
│   ┌─────────────────────────────────────────────────────────────┐   │
│   │                      Amazon Redshift                         │   │
│   │                    (Data Warehouse)                          │   │
│   │                                                              │   │
│   │  Historical Analytics  •  Business Intelligence             │   │
│   │  Reporting            •  Machine Learning Training          │   │
│   └─────────────────────────────────────────────────────────────┘   │
│                              │                       Cold Data      │
│                              ▼                                       │
│   ┌─────────────────────────────────────────────────────────────┐   │
│   │                      Amazon S3                               │   │
│   │                  (Data Lake / Archive)                       │   │
│   └─────────────────────────────────────────────────────────────┘   │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```